In [1]:
import pandas as pd
import numpy as np

# Simulate user IDs
user_ids = np.arange(1000)

# Simulate events with typical drop-offs
data = {
    'user_id': user_ids,
    'step_1_visited_homepage': np.where(user_ids < 1000, 1, 0), # 1000 users start
    'step_2_clicked_pricing': np.where(user_ids < 800, 1, 0),    # 20% drop-off
    'step_3_signed_up': np.where(user_ids < 600, 1, 0),          # 25% drop-off
    'step_4_completed_onboarding': np.where(user_ids < 480, 1, 0) # 20% drop-off
}

df_events = pd.DataFrame(data)
df_events.to_csv('user_events.csv', index=False)
print("user_events.csv created!")

user_events.csv created!


In [2]:
import pandas as pd

def funnel_analyze(file_path):
    """
    Analyzes a user event funnel from a CSV file.
    Assumes columns are sequential steps (e.g., step_1, step_2).
    """
    df = pd.read_csv(file_path)
    # Identify funnel steps (columns starting with 'step_')
    steps = [col for col in df.columns if col.startswith('step_')]

    if not steps:
        print("Error: No funnel steps found in the file.")
        return

    # Calculate starting users (Total)
    total_starters = df[steps[0]].sum()

    funnel_data = []
    previous_count = 0

    for i, step in enumerate(steps):
        current_count = df[step].sum()

        if i == 0:
            step_conversion = 1.0 # 100% conversion from 'start'
            overall_conversion = 1.0
        else:
            # Step-to-step conversion
            step_conversion = current_count / previous_count if previous_count > 0 else 0
            # Overall conversion (from step 1)
            overall_conversion = current_count / total_starters if total_starters > 0 else 0

        funnel_data.append({
            'Step': step,
            'Users': current_count,
            'Step Conv. Rate': f"{step_conversion * 100:.2f}%",
            'Overall Conv. Rate': f"{overall_conversion * 100:.2f}%"
        })

        previous_count = current_count

    # Display results
    print("\n--- 📈 Funnel Analysis Results ---")
    results_df = pd.DataFrame(funnel_data)

    # Calculate drop-off
    results_df['Drop-off'] = results_df['Users'].diff().fillna(0).astype(int) * -1
    results_df['Drop-off'] = results_df['Drop-off'].apply(lambda x: 0 if x < 0 else x)

    # Format and print the final table
    print(results_df[['Step', 'Users', 'Drop-off', 'Step Conv. Rate', 'Overall Conv. Rate']].to_markdown(index=False))

# --- Run the function with the sample data ---
funnel_analyze('user_events.csv')


--- 📈 Funnel Analysis Results ---
| Step                        |   Users |   Drop-off | Step Conv. Rate   | Overall Conv. Rate   |
|:----------------------------|--------:|-----------:|:------------------|:---------------------|
| step_1_visited_homepage     |    1000 |          0 | 100.00%           | 100.00%              |
| step_2_clicked_pricing      |     800 |        200 | 80.00%            | 80.00%               |
| step_3_signed_up            |     600 |        200 | 75.00%            | 60.00%               |
| step_4_completed_onboarding |     480 |        120 | 80.00%            | 48.00%               |
